# The SubClassing API

In [1]:
from sklearn.datasets import fetch_california_housing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
from tensorflow import keras

In [2]:
# loading and splitting the dataset into train, valid and test set
housing = fetch_california_housing()

X_train_full, X_test, y_train_full, y_test = train_test_split(housing.data, housing.target)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full)

In [3]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_valid = scaler.fit_transform(X_valid)
X_test = scaler.fit_transform(X_test)

In [6]:
X_train_A, X_train_B = X_train[:, :5], X_train[:, 2:]
X_valid_A, X_valid_B = X_valid[:, :5], X_valid[:, 2:]
X_test_A, X_test_B = X_test[:, :5], X_test[:, 2:]
X_new_A, X_new_B = X_test_A[:3], X_test_B[:3]

- Both the Sequential API and the Functional API are declarative: you start by declaring which layers you want to use and how they should be connected, and only then can you start feeding the model some data for training or inference.

- This has many advantages: the model can easily be saved, cloned, and shared; its structure can be displayed and analyzed; the framework can infer shapes and check types, so errors can be caught early (i.e., before any data ever goes through the model). It’s also fairly easy to debug, since the whole model is a static graph of layers. 

- But the flip side is just that: it’s static. Some models involve loops, varying shapes, conditional branching, and other dynamic behaviors. For such cases, or simply if you prefer a more imperative programming style, the Subclassing API is for you. 

- Simply subclass the Model class, create the layers you need in the constructor, and use them to perform the computations you want in the call() method. For example, creating an instance of the following WideAndDeepModel class gives us an equivalent model to the one we just built with the Functional API. You can then compile it, evaluate it, and use it to make predictions, exactly like we just did [here](https://github.com/Gladiator07/Deep-Learning/blob/main/Notebooks/Functional_API.ipynb): 

In [4]:
class WideAndDeepModel(keras.Model):
    def __init__(self, units=30, activation="relu", **kwargs):
        super().__init__(**kwargs) # handles standard args (e.g., name)
        self.hidden1 = keras.layers.Dense(units, activation=activation)
        self.hidden2 = keras.layers.Dense(units, activation=activation)
        self.main_output = keras.layers.Dense(1)
        self.aux_output = keras.layers.Dense(1)

    def call(self, inputs):
        input_A, input_B = inputs
        hidden1 = self.hidden1(input_B)
        hidden2 = self.hidden2(hidden1)
        concat = keras.layers.concatenate([input_A, hidden2])
        main_output = self.main_output(concat)
        aux_output = self.aux_output(hidden2)
        return main_output, aux_output

In [5]:
model = WideAndDeepModel(30, activation="relu")

In [7]:
model.compile(loss="mse", loss_weights=[0.9, 0.1], optimizer=keras.optimizers.SGD(lr=1e-3))
history = model.fit((X_train_A, X_train_B), (y_train, y_train), epochs=10,
                    validation_data=((X_valid_A, X_valid_B), (y_valid, y_valid)))
total_loss, main_loss, aux_loss = model.evaluate((X_test_A, X_test_B), (y_test, y_test))
y_pred_main, y_pred_aux = model.predict((X_new_A, X_new_B))

Epoch 1/10
363/363 [==============================] - 1s 4ms/step - loss: 2.6147 - output_1_loss: 2.5023 - output_2_loss: 3.6257 - val_loss: 1.2881 - val_output_1_loss: 1.1049 - val_output_2_loss: 2.9370
Epoch 2/10
363/363 [==============================] - 1s 3ms/step - loss: 1.0461 - output_1_loss: 0.8862 - output_2_loss: 2.4855 - val_loss: 0.9388 - val_output_1_loss: 0.8090 - val_output_2_loss: 2.1071
Epoch 3/10
363/363 [==============================] - 1s 3ms/step - loss: 0.8388 - output_1_loss: 0.7178 - output_2_loss: 1.9275 - val_loss: 0.8228 - val_output_1_loss: 0.7198 - val_output_2_loss: 1.7494
Epoch 4/10
363/363 [==============================] - 1s 3ms/step - loss: 0.7572 - output_1_loss: 0.6548 - output_2_loss: 1.6782 - val_loss: 0.7656 - val_output_1_loss: 0.6753 - val_output_2_loss: 1.5790
Epoch 5/10
363/363 [==============================] - 1s 3ms/step - loss: 0.7134 - output_1_loss: 0.6206 - output_2_loss: 1.5484 - val_loss: 0.7287 - val_output_1_loss: 0.6448 - val_ou

This example looks very much like the Functional API, except we do not need to create the inputs; we just use the input argument to the call() method, and we separate the creation of the layers21 in the constructor from their usage in the call() method. 

The big difference is that you can do pretty much anything you want in the call() method: for loops, if statements, low-level TensorFlow operations—your imagina‐ tion is the limit. This makes it a great API for researchers experi‐ menting with new ideas. 

This extra flexibility does come at a cost: your model’s architecture is hidden within the call() method, so Keras cannot easily inspect it; it cannot save or clone it; and when you call the summary() method, you only get a list of layers, without any information on how they are connected to each other. 

Moreover, Keras cannot check types and shapes ahead of time, and it is easier to make mistakes. So unless you really need that extra flexibility, you should probably stick to the Sequential API or the Functional API.


## Saving and Restoring Model for Dynamic Models

In [9]:
# model.save("my_keras_model.h5")

#### This statement will give error as, Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.